# Event and Venue Data Extractor

This notebook extracts unique event and venue data from the CONCERT_SEATS table and inserts it into the EVENTS and VENUES tables.

## Features:
- ✅ Extract unique venues with city and country data
- ✅ Extract unique events with all related information  
- ✅ Duplicate prevention using INSERT IGNORE
- ✅ Data quality checks and validation
- ✅ Batch processing for large datasets
- ✅ Progress tracking and statistics

## Table of Contents:
1. [Setup & Database Connection](#setup--database-connection)
2. [Venue Data Extraction](#venue-data-extraction)
3. [Event Data Extraction](#event-data-extraction)
4. [Complete Extraction Process](#complete-extraction-process)
5. [Statistics & Verification](#statistics--verification)

## Setup & Database Connection

### Import Required Libraries and Load Database Configuration

In [1]:
# Import required libraries
import json
import mysql.connector
from mysql.connector import Error
import pandas as pd
from datetime import datetime
import time
import sys

print("✅ All libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All libraries imported successfully!
📅 Current date: 2025-07-20 15:08:46


In [2]:
# Load database configuration
try:
    with open('db.json', 'r') as f:
        db_config = json.load(f)
    
    print("✅ Database configuration loaded:")
    print(f"   Host: {db_config['host']}")
    print(f"   User: {db_config['user']}")
    print(f"   Database: {db_config['database']}")
    print("   Password: [HIDDEN]")
except FileNotFoundError:
    print("❌ db.json file not found. Please ensure it exists in the current directory.")
    db_config = None
except Exception as e:
    print(f"❌ Error loading database configuration: {e}")
    db_config = None

✅ Database configuration loaded:
   Host: 192.168.68.74
   User: root
   Database: concert
   Password: [HIDDEN]


In [3]:
# Establish database connection
connection = None
cursor = None

try:
    connection = mysql.connector.connect(**db_config)
    
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"✅ Connected to MySQL Server version {db_info}")
        
        cursor = connection.cursor()
        
        # Get database name
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"✅ Connected to database: {record[0]}")
        
except Error as e:
    print(f"❌ Database connection failed: {e}")
    connection = None
    cursor = None

✅ Connected to MySQL Server version 5.5.5-10.6.21-MariaDB-ubu2004-log
✅ Connected to database: concert


C:\Users\eric9\AppData\Local\Temp\ipykernel_22668\4062101452.py:9: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()


### Database Utility Functions

In [4]:
def close_connection():
    """Close database connection properly."""
    global connection, cursor
    
    try:
        if cursor:
            cursor.close()
            print("✓ Cursor closed.")
        
        if connection and connection.is_connected():
            connection.close()
            print("✓ MySQL connection closed.")
    except Error as e:
        print(f"❌ Error closing connection: {e}")

def reconnect_if_needed():
    """Reconnect to database if connection is lost."""
    global connection, cursor
    
    if not connection or not connection.is_connected():
        print("🔄 Reconnecting to database...")
        try:
            connection = mysql.connector.connect(**db_config)
            cursor = connection.cursor()
            print("✅ Reconnected successfully!")
            return True
        except Error as e:
            print(f"❌ Reconnection failed: {e}")
            return False
    return True

print("✅ Database utility functions defined!")

✅ Database utility functions defined!


## Venue Data Extraction

### Extract Unique Venues from CONCERT_SEATS Table

In [5]:
# Extract unique venue data from CONCERT_SEATS table
print("🏟️ Extracting venue data from CONCERT_SEATS table...")

venue_query = """
    SELECT DISTINCT 
        venue as name,
        city,
        countryName as country
    FROM CONCERT_SEATS 
    WHERE venue IS NOT NULL 
        AND venue != '' 
        AND venue != 'None'
        AND venue != 'NULL'
    ORDER BY venue
"""

try:
    reconnect_if_needed()
    venues_df = pd.read_sql(venue_query, connection)
    print(f"✅ Extracted {len(venues_df)} unique venues")
    
    # Display sample venues
    print("\n📋 Sample venues:")
    display(venues_df.head(10))
    
except Error as e:
    print(f"❌ Error extracting venue data: {e}")
    venues_df = pd.DataFrame()

🏟️ Extracting venue data from CONCERT_SEATS table...


C:\Users\eric9\AppData\Local\Temp\ipykernel_22668\1150478942.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  venues_df = pd.read_sql(venue_query, connection)


✅ Extracted 2017 unique venues

📋 Sample venues:


,name,city,country
0,10 Mile Music Hall,Frisco,USA
1,1015 Folsom,San Francisco,USA
2,11:11 EPTX,El Paso,USA
3,1614 Drinks-Music-Billiards / AfterHours Tavern,High Point,USA
4,1902 Nightclub,San Antonio,USA
5,191 Toole,Tucson,USA
6,3 Dollar Bill,Brooklyn,USA
7,3rd and Lindsley Bar & Grill,Nashville,USA
8,45 East,Portland,USA
9,4th Street Live,Louisville,USA


In [6]:
# Check for data quality issues in venues
if not venues_df.empty:
    print("🔍 Checking venue data quality...")
    
    # Check for venues with multiple cities (potential data quality issues)
    venue_cities = venues_df.groupby('name').agg({
        'city': lambda x: list(set(x.dropna())),
        'country': lambda x: list(set(x.dropna()))
    }).reset_index()
    
    duplicates_with_diff_cities = venue_cities[
        venue_cities['city'].apply(len) > 1
    ]
    
    if len(duplicates_with_diff_cities) > 0:
        print(f"⚠️  Found {len(duplicates_with_diff_cities)} venues with multiple cities:")
        for _, row in duplicates_with_diff_cities.head().iterrows():
            print(f"   • {row['name']}: {row['city']}")
    else:
        print("✅ No venues with conflicting city data found")
    
    # Show venue statistics
    print(f"\n📊 Venue Statistics:")
    print(f"   Total unique venues: {len(venues_df)}")
    print(f"   Countries represented: {venues_df['country'].nunique()}")
    print(f"   Cities represented: {venues_df['city'].nunique()}")
    
    # Top countries by venue count
    top_countries = venues_df['country'].value_counts().head(5)
    print(f"\n🌍 Top countries by venue count:")
    for country, count in top_countries.items():
        print(f"   • {country}: {count} venues")

🔍 Checking venue data quality...
⚠️  Found 2 venues with multiple cities:
   • Budweiser Stage at Ontario Place - Complex: ['London', 'Toronto']
   • The Dome: ['Halifax', 'Virginia Beach']

📊 Venue Statistics:
   Total unique venues: 2017
   Countries represented: 11
   Cities represented: 694

🌍 Top countries by venue count:
   • USA: 1743 venues
   • Canada: 261 venues
   • France: 2 venues
   • Mexico: 2 venues
   • UK: 2 venues
⚠️  Found 2 venues with multiple cities:
   • Budweiser Stage at Ontario Place - Complex: ['London', 'Toronto']
   • The Dome: ['Halifax', 'Virginia Beach']

📊 Venue Statistics:
   Total unique venues: 2017
   Countries represented: 11
   Cities represented: 694

🌍 Top countries by venue count:
   • USA: 1743 venues
   • Canada: 261 venues
   • France: 2 venues
   • Mexico: 2 venues
   • UK: 2 venues


In [7]:
# Insert venue data into VENUES table
print("🔄 Inserting venue data into VENUES table...")

if not venues_df.empty:
    try:
        reconnect_if_needed()
        
        # Prepare venue data for insertion
        venue_data = []
        for _, row in venues_df.iterrows():
            venue_data.append((
                row['name'] if pd.notna(row['name']) else None,
                row['city'] if pd.notna(row['city']) else None,
                row['country'] if pd.notna(row['country']) else None
            ))
        
        # Insert venues using INSERT IGNORE to avoid duplicates
        venue_insert_query = """
            INSERT IGNORE INTO VENUES (name, city, country) 
            VALUES (%s, %s, %s)
        """
        
        batch_size = 1000
        total_inserted = 0
        
        for i in range(0, len(venue_data), batch_size):
            batch = venue_data[i:i + batch_size]
            cursor.executemany(venue_insert_query, batch)
            connection.commit()
            total_inserted += cursor.rowcount
            print(f"   Processed {min(i + batch_size, len(venue_data))}/{len(venue_data)} venues...")
        
        print(f"✅ Successfully inserted {total_inserted} new venues into VENUES table")
        
        # Verify insertion
        cursor.execute("SELECT COUNT(*) FROM VENUES")
        total_venues = cursor.fetchone()[0]
        print(f"📊 Total venues in VENUES table: {total_venues}")
        
    except Error as e:
        print(f"❌ Error inserting venue data: {e}")
        if connection:
            connection.rollback()
else:
    print("❌ No venue data to insert")

🔄 Inserting venue data into VENUES table...
   Processed 1000/2017 venues...
   Processed 2000/2017 venues...
   Processed 2017/2017 venues...
✅ Successfully inserted 245 new venues into VENUES table
📊 Total venues in VENUES table: 2015
   Processed 1000/2017 venues...
   Processed 2000/2017 venues...
   Processed 2017/2017 venues...
✅ Successfully inserted 245 new venues into VENUES table
📊 Total venues in VENUES table: 2015


## Event Data Extraction

### Extract Unique Events from CONCERT_SEATS Table

In [8]:
# Extract unique event data from CONCERT_SEATS table
print("🎪 Extracting event data from CONCERT_SEATS table...")

event_query = """
    SELECT DISTINCT 
        eventId as event_id,
        venue as venue_id,
        artist,
        event_name,
        event_date,
        event_type,
        performer_type,
        performer
    FROM CONCERT_SEATS 
    WHERE eventId IS NOT NULL 
        AND eventId != 0
        AND venue IS NOT NULL 
        AND venue != '' 
        AND venue != 'None'
        AND venue != 'NULL'
        AND artist IS NOT NULL 
        AND artist != '' 
        AND artist != 'None'
        AND artist != 'NULL'
    ORDER BY eventId
"""

try:
    reconnect_if_needed()
    events_df = pd.read_sql(event_query, connection)
    print(f"✅ Extracted {len(events_df)} unique events")
    
    # Display sample events
    print("\n📋 Sample events:")
    display(events_df.head(10))
    
except Error as e:
    print(f"❌ Error extracting event data: {e}")
    events_df = pd.DataFrame()

🎪 Extracting event data from CONCERT_SEATS table...


C:\Users\eric9\AppData\Local\Temp\ipykernel_22668\4101512482.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events_df = pd.read_sql(event_query, connection)


✅ Extracted 6772 unique events

📋 Sample events:


,event_id,venue_id,artist,event_name,event_date,event_type,performer_type,performer
0,152891943.0,Greek Theatre Los Angeles,Omd,OMD,2025-06-21,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
1,153021829.0,Greek Theatre Los Angeles,Omd,OMD,2025-06-20,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
2,153112606.0,The Paramount Huntington,Omd,OMD,2025-07-06,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
3,153112667.0,Keswick Theatre,Omd,OMD,2025-05-24,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
4,153112669.0,Terminal 5,Omd,OMD,2025-07-08,None,None,None
5,153112669.0,Terminal 5,Omd,OMD,2025-07-08,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
6,153112768.0,Riviera Theatre Chicago,Omd,OMD,2025-06-28,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
7,153113241.0,Balboa Theatre,Omd,OMD,2025-06-17,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
8,153113243.0,Lincoln Theatre DC,Omd,OMD,2025-05-22,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark
9,153114494.0,Citizens House of Blues Boston,Omd,OMD,2025-07-07,MusicEvent,MusicGroup,Orchestral Manoeuvres In The Dark


In [9]:
# Check event data quality and statistics
if not events_df.empty:
    print("🔍 Analyzing event data quality...")
    
    # Check for events with missing data
    missing_data_count = events_df.isnull().sum()
    if missing_data_count.any():
        print("\n⚠️  Missing data summary:")
        for col, count in missing_data_count.items():
            if count > 0:
                print(f"   • {col}: {count} missing values ({count/len(events_df)*100:.1f}%)")
    else:
        print("✅ No missing data found in events")
    
    # Show event statistics
    print(f"\n📊 Event Statistics:")
    print(f"   Total unique events: {len(events_df)}")
    print(f"   Unique artists: {events_df['artist'].nunique()}")
    print(f"   Unique venues: {events_df['venue_id'].nunique()}")
    print(f"   Date range: {events_df['event_date'].min()} to {events_df['event_date'].max()}")
    
    # Top artists by event count
    top_artists = events_df['artist'].value_counts().head(5)
    print(f"\n🎤 Top artists by event count:")
    for artist, count in top_artists.items():
        print(f"   • {artist}: {count} events")
    
    # Top venues by event count
    top_venues = events_df['venue_id'].value_counts().head(5)
    print(f"\n🏟️  Top venues by event count:")
    for venue, count in top_venues.items():
        print(f"   • {venue}: {count} events")

🔍 Analyzing event data quality...

⚠️  Missing data summary:
   • event_type: 86 missing values (1.3%)
   • performer_type: 86 missing values (1.3%)
   • performer: 86 missing values (1.3%)

📊 Event Statistics:
   Total unique events: 6772
   Unique artists: 487
   Unique venues: 2014
   Date range: 2025-02-20 to 2026-09-10

🎤 Top artists by event count:
   • Under Oath: 50 events
   • Nao: 49 events
   • Shawn Desman: 45 events
   • Subtronics: 44 events
   • Westend: 44 events

🏟️  Top venues by event count:
   • History: 98 events
   • The Brooklyn Mirage at Avant Gardner - Complex: 61 events
   • Budweiser Stage at Ontario Place - Complex: 58 events
   • Rogers Centre: 56 events
   • Scotiabank Arena: 47 events


In [10]:
# Insert event data into EVENTS table
print("🔄 Inserting event data into EVENTS table...")

if not events_df.empty:
    try:
        reconnect_if_needed()
        
        # Prepare event data for insertion
        event_data = []
        for _, row in events_df.iterrows():
            event_data.append((
                int(row['event_id']) if pd.notna(row['event_id']) else None,
                row['venue_id'] if pd.notna(row['venue_id']) else None,
                row['artist'] if pd.notna(row['artist']) else None,
                row['event_name'] if pd.notna(row['event_name']) else None,
                row['event_date'] if pd.notna(row['event_date']) else None,
                row['event_type'] if pd.notna(row['event_type']) else None,
                row['performer_type'] if pd.notna(row['performer_type']) else None,
                row['performer'] if pd.notna(row['performer']) else None
            ))
        
        # Insert events using INSERT IGNORE to avoid duplicates
        event_insert_query = """
            INSERT IGNORE INTO EVENTS (
                event_id, venue_id, artist, event_name, event_date,
                event_type, performer_type, performer
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        batch_size = 1000
        total_inserted = 0
        
        for i in range(0, len(event_data), batch_size):
            batch = event_data[i:i + batch_size]
            cursor.executemany(event_insert_query, batch)
            connection.commit()
            total_inserted += cursor.rowcount
            print(f"   Processed {min(i + batch_size, len(event_data))}/{len(event_data)} events...")
        
        print(f"✅ Successfully inserted {total_inserted} new events into EVENTS table")
        
        # Verify insertion
        cursor.execute("SELECT COUNT(*) FROM EVENTS")
        total_events = cursor.fetchone()[0]
        print(f"📊 Total events in EVENTS table: {total_events}")
        
    except Error as e:
        print(f"❌ Error inserting event data: {e}")
        if connection:
            connection.rollback()
else:
    print("❌ No event data to insert")

🔄 Inserting event data into EVENTS table...
❌ Error inserting event data: 1054 (42S22): Unknown column 'venue_id' in 'INSERT INTO'
❌ Error inserting event data: 1054 (42S22): Unknown column 'venue_id' in 'INSERT INTO'


## Complete Extraction Process

### Run Full Extraction with Timing

In [11]:
# Complete extraction process with timing
def run_complete_extraction():
    """Run the complete extraction and insertion process with timing."""
    
    start_time = datetime.now()
    print("🚀 Starting Complete Event and Venue Data Extraction Process")
    print(f"📅 Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60)
    
    extraction_stats = {
        'venues_extracted': 0,
        'venues_inserted': 0,
        'events_extracted': 0,
        'events_inserted': 0,
        'success': True
    }
    
    try:
        # Ensure connection is active
        if not reconnect_if_needed():
            return False
        
        # Extract and insert venue data
        print("\n" + "="*25 + " VENUES " + "="*25)
        venue_start = time.time()
        
        # Re-extract venues (in case data changed)
        venues_df_new = pd.read_sql(venue_query, connection)
        extraction_stats['venues_extracted'] = len(venues_df_new)
        print(f"✅ Extracted {len(venues_df_new)} venues")
        
        if not venues_df_new.empty:
            # Insert venues
            venue_data = [(row['name'], row['city'], row['country']) 
                         for _, row in venues_df_new.iterrows()]
            
            cursor.executemany(venue_insert_query, venue_data)
            connection.commit()
            extraction_stats['venues_inserted'] = cursor.rowcount
            
        venue_time = time.time() - venue_start
        print(f"⏱️  Venue processing completed in {venue_time:.2f} seconds")
        
        # Extract and insert event data
        print("\n" + "="*25 + " EVENTS " + "="*25)
        event_start = time.time()
        
        # Re-extract events (in case data changed)
        events_df_new = pd.read_sql(event_query, connection)
        extraction_stats['events_extracted'] = len(events_df_new)
        print(f"✅ Extracted {len(events_df_new)} events")
        
        if not events_df_new.empty:
            # Insert events
            event_data = [
                (int(row['event_id']), row['venue_id'], row['artist'], 
                 row['event_name'], row['event_date'], row['event_type'],
                 row['performer_type'], row['performer'])
                for _, row in events_df_new.iterrows()
            ]
            
            cursor.executemany(event_insert_query, event_data)
            connection.commit()
            extraction_stats['events_inserted'] = cursor.rowcount
            
        event_time = time.time() - event_start
        print(f"⏱️  Event processing completed in {event_time:.2f} seconds")
        
        # Final summary
        end_time = datetime.now()
        total_duration = end_time - start_time
        
        print("\n" + "="*60)
        print("🎯 COMPLETE EXTRACTION PROCESS FINISHED!")
        print("✅ All operations completed successfully!")
        print(f"\n📊 Extraction Summary:")
        print(f"   🏟️  Venues: {extraction_stats['venues_extracted']} extracted, {extraction_stats['venues_inserted']} new inserted")
        print(f"   🎪 Events: {extraction_stats['events_extracted']} extracted, {extraction_stats['events_inserted']} new inserted")
        print(f"\n⏱️  Total execution time: {total_duration}")
        print(f"📅 Completed at: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
        
        return True
        
    except Exception as e:
        print(f"❌ Unexpected error during extraction: {e}")
        extraction_stats['success'] = False
        return False

# Run the complete extraction
run_complete_extraction()

🚀 Starting Complete Event and Venue Data Extraction Process
📅 Started at: 2025-07-20 15:10:20

========================= VENUES =========================


C:\Users\eric9\AppData\Local\Temp\ipykernel_22668\781228854.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  venues_df_new = pd.read_sql(venue_query, connection)


✅ Extracted 2017 venues
⏱️  Venue processing completed in 28.29 seconds

========================= EVENTS =========================
⏱️  Venue processing completed in 28.29 seconds

========================= EVENTS =========================


C:\Users\eric9\AppData\Local\Temp\ipykernel_22668\781228854.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events_df_new = pd.read_sql(event_query, connection)


✅ Extracted 6772 events
❌ Unexpected error during extraction: 1054 (42S22): Unknown column 'venue_id' in 'INSERT INTO'
❌ Unexpected error during extraction: 1054 (42S22): Unknown column 'venue_id' in 'INSERT INTO'


False

## Statistics & Verification

### Show Final Database Statistics

In [12]:
# Show comprehensive database statistics
def show_comprehensive_stats():
    """Show detailed statistics for EVENTS and VENUES tables."""
    
    try:
        reconnect_if_needed()
        
        print("📊 Comprehensive Database Statistics")
        print("="*50)
        
        # VENUES detailed stats
        cursor.execute("SELECT COUNT(*) FROM VENUES")
        venue_count = cursor.fetchone()[0]
        print(f"\n🏟️  VENUES TABLE: {venue_count} records")
        
        if venue_count > 0:
            # Top countries
            cursor.execute("""
                SELECT country, COUNT(*) as count 
                FROM VENUES 
                WHERE country IS NOT NULL 
                GROUP BY country 
                ORDER BY count DESC 
                LIMIT 10
            """)
            countries = cursor.fetchall()
            print("   📍 Countries with most venues:")
            for country, count in countries:
                print(f"      • {country}: {count} venues")
            
            # Sample venues
            cursor.execute("SELECT name, city, country FROM VENUES ORDER BY name LIMIT 5")
            sample_venues = cursor.fetchall()
            print("   \n🏟️  Sample venues:")
            for name, city, country in sample_venues:
                print(f"      • {name} ({city}, {country})")
        
        # EVENTS detailed stats  
        cursor.execute("SELECT COUNT(*) FROM EVENTS")
        event_count = cursor.fetchone()[0]
        print(f"\n🎪 EVENTS TABLE: {event_count} records")
        
        if event_count > 0:
            # Top artists
            cursor.execute("""
                SELECT artist, COUNT(*) as count 
                FROM EVENTS 
                WHERE artist IS NOT NULL 
                GROUP BY artist 
                ORDER BY count DESC 
                LIMIT 10
            """)
            artists = cursor.fetchall()
            print("   🎤 Artists with most events:")
            for artist, count in artists:
                print(f"      • {artist}: {count} events")
            
            # Date range
            cursor.execute("""
                SELECT MIN(event_date) as earliest, MAX(event_date) as latest 
                FROM EVENTS 
                WHERE event_date IS NOT NULL
            """)
            date_range = cursor.fetchone()
            if date_range[0] and date_range[1]:
                print(f"   📅 Date range: {date_range[0]} to {date_range[1]}")
            
            # Event types
            cursor.execute("""
                SELECT event_type, COUNT(*) as count 
                FROM EVENTS 
                WHERE event_type IS NOT NULL 
                GROUP BY event_type 
                ORDER BY count DESC 
                LIMIT 5
            """)
            event_types = cursor.fetchall()
            print("   🎭 Event types:")
            for event_type, count in event_types:
                print(f"      • {event_type}: {count} events")
        
        # Cross-table statistics
        cursor.execute("""
            SELECT COUNT(DISTINCT e.venue_id) as venues_with_events
            FROM EVENTS e
            WHERE e.venue_id IS NOT NULL
        """)
        venues_with_events = cursor.fetchone()[0]
        
        print(f"\n🔗 Cross-table Statistics:")
        print(f"   📊 Venues with events: {venues_with_events}")
        
        if venue_count > 0:
            utilization = (venues_with_events / venue_count) * 100
            print(f"   📈 Venue utilization: {utilization:.1f}%")
        
    except Error as e:
        print(f"❌ Error retrieving comprehensive statistics: {e}")

# Show the comprehensive statistics
show_comprehensive_stats()

📊 Comprehensive Database Statistics

🏟️  VENUES TABLE: 2015 records
   📍 Countries with most venues:
      • USA: 1742 venues
      • Canada: 260 venues
      • France: 2 venues
      • UK: 2 venues
      • Germany: 2 venues
      • Mexico: 2 venues
      • Brazil: 1 venues
      • Netherlands: 1 venues
      • Switzerland: 1 venues
      • Norway: 1 venues
   
🏟️  Sample venues:
      • 10 Mile Music Hall (Frisco, USA)
      • 1015 Folsom (San Francisco, USA)
      • 11:11 EPTX (El Paso, USA)
      • 1614 Drinks-Music-Billiards / AfterHours Tavern (High Point, USA)
      • 1902 Nightclub (San Antonio, USA)

🎪 EVENTS TABLE: 4778 records
   🎤 Artists with most events:
      • 49Th & Main: 39 events
      • Shawn Desman: 37 events
      • Bunt: 36 events
      • Crankdat: 36 events
      • Nao: 36 events
      • Knock2 : 33 events
      • Goo Goo: 32 events
      • Westend: 32 events
      • Shaq: 31 events
      • Under Oath: 30 events
   🎤 Artists with most events:
      • 49Th & Main:

In [13]:
# Verification queries - check data integrity
print("🔍 Data Integrity Verification")
print("="*40)

try:
    reconnect_if_needed()
    
    # Check for orphaned events (events with venues not in VENUES table)
    cursor.execute("""
        SELECT COUNT(*) 
        FROM EVENTS e 
        LEFT JOIN VENUES v ON e.venue_id = v.name 
        WHERE v.name IS NULL AND e.venue_id IS NOT NULL
    """)
    orphaned_events = cursor.fetchone()[0]
    
    if orphaned_events > 0:
        print(f"⚠️  Found {orphaned_events} events with venues not in VENUES table")
        
        # Show sample orphaned events
        cursor.execute("""
            SELECT e.event_id, e.venue_id, e.artist, e.event_name 
            FROM EVENTS e 
            LEFT JOIN VENUES v ON e.venue_id = v.name 
            WHERE v.name IS NULL AND e.venue_id IS NOT NULL 
            LIMIT 5
        """)
        orphaned_sample = cursor.fetchall()
        print("   Sample orphaned events:")
        for event_id, venue, artist, event_name in orphaned_sample:
            print(f"      • Event {event_id}: {artist} at {venue}")
    else:
        print("✅ All events have corresponding venues in VENUES table")
    
    # Check for duplicate event IDs
    cursor.execute("""
        SELECT event_id, COUNT(*) as count 
        FROM EVENTS 
        GROUP BY event_id 
        HAVING COUNT(*) > 1
    """)
    duplicate_events = cursor.fetchall()
    
    if duplicate_events:
        print(f"⚠️  Found {len(duplicate_events)} duplicate event IDs:")
        for event_id, count in duplicate_events[:5]:
            print(f"      • Event ID {event_id}: {count} duplicates")
    else:
        print("✅ No duplicate event IDs found")
    
    # Check for venues without events
    cursor.execute("""
        SELECT COUNT(*) 
        FROM VENUES v 
        LEFT JOIN EVENTS e ON v.name = e.venue_id 
        WHERE e.venue_id IS NULL
    """)
    venues_without_events = cursor.fetchone()[0]
    
    print(f"📊 Found {venues_without_events} venues without events (this is normal)")
    
except Error as e:
    print(f"❌ Error during verification: {e}")

🔍 Data Integrity Verification
❌ Error during verification: 1054 (42S22): Unknown column 'e.venue_id' in 'WHERE'


## Database Cleanup

### Close Database Connection

In [14]:
# Final cleanup and summary
print("🧹 Final Cleanup and Summary")
print("="*40)

# Show final counts
try:
    if connection and connection.is_connected():
        cursor.execute("SELECT COUNT(*) FROM VENUES")
        final_venue_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM EVENTS") 
        final_event_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM CONCERT_SEATS")
        seat_count = cursor.fetchone()[0]
        
        print(f"📊 Final Database Summary:")
        print(f"   🏟️  VENUES table: {final_venue_count} records")
        print(f"   🎪 EVENTS table: {final_event_count} records")
        print(f"   🎫 CONCERT_SEATS table: {seat_count} records")
        print(f"\n✅ Data extraction and normalization completed successfully!")
        
except Error as e:
    print(f"❌ Error getting final counts: {e}")

# Close the database connection
close_connection()
print("\n🎯 Notebook execution completed!")

🧹 Final Cleanup and Summary
📊 Final Database Summary:
   🏟️  VENUES table: 2015 records
   🎪 EVENTS table: 4778 records
   🎫 CONCERT_SEATS table: 192126 records

✅ Data extraction and normalization completed successfully!
✓ Cursor closed.
✓ MySQL connection closed.

🎯 Notebook execution completed!
📊 Final Database Summary:
   🏟️  VENUES table: 2015 records
   🎪 EVENTS table: 4778 records
   🎫 CONCERT_SEATS table: 192126 records

✅ Data extraction and normalization completed successfully!
✓ Cursor closed.
✓ MySQL connection closed.

🎯 Notebook execution completed!


---

## Summary

This notebook provides a complete workflow for extracting and normalizing event and venue data:

### ✅ **Completed Tasks:**
1. **Database Connection** - Established connection with error handling
2. **Venue Extraction** - Extracted unique venues with location data
3. **Event Extraction** - Extracted unique events with full details
4. **Data Quality Checks** - Validated data integrity and identified issues
5. **Batch Insertion** - Efficiently inserted data with duplicate prevention
6. **Statistics & Verification** - Comprehensive analysis and validation

### 🎯 **Key Features:**
- **Smart Duplicate Handling** - Uses INSERT IGNORE to prevent duplicates
- **Data Quality Validation** - Identifies and reports data inconsistencies
- **Progress Tracking** - Shows detailed progress and timing information
- **Error Handling** - Comprehensive error handling with rollback capability
- **Interactive Analysis** - Step-by-step execution with detailed statistics

### 📊 **Data Flow:**
```
CONCERT_SEATS → Extract Unique Data → VENUES + EVENTS Tables
```

### 🚀 **Next Steps:**
- Run cells sequentially from top to bottom
- Monitor progress and statistics in each section  
- Use the verification section to ensure data integrity
- Customize queries as needed for your specific requirements